In [1]:
"""
Plan:
Quantum k-maxima finding algorithm:
  1. Set A = k indices, randomly chosen
  2. Choose an index y from the indices featuring in set A.
  3. Using Quantum Search Algorithm find an index Y such that FY > fy and Y is not in Set A.
     - Modified Grover Algorithms
  4. Replace y with Y in Set A.
  5. Repeat until K nearest neighbours found.
  We can map coordinates on previous KNN to unique strings which we can run grovers on.
Ref:
https://www.geeksforgeeks.org/introduction-to-grovers-algorithm/
https://arxiv.org/pdf/2003.09187
https://medium.com/quantum-untangled/grovers-algorithm-mathematics-circuits-and-code-quantum-algorithms-untangled-c4aa47d506e5
CP351 Notes
ChatGPT
"""

'\nPlan:\nQuantum k-maxima finding algorithm:\n  1. Set A = k indices, randomly chosen\n  2. Choose an index y from the indices featuring in set A.\n  3. Using Quantum Search Algorithm find an index Y such that FY > fy and Y is not in Set A.\n     - Modified Grover Algorithms\n  4. Replace y with Y in Set A.\n  5. Repeat until K nearest neighbours found.\n  We can map coordinates on previous KNN to unique strings which we can run grovers on.\nRef:\nhttps://www.geeksforgeeks.org/introduction-to-grovers-algorithm/\nhttps://arxiv.org/pdf/2003.09187\nCP351 Notes\nChatGPT\n'

In [2]:
# Python Set, unordered, unindexed data type, every item is unique.
import random
k = 3
Mcoordinates = [
    "x:08,y:26",
    "x:11,y:21",
    "x:07,y:18",
    "x:14,y:25",
    "x:13,y:28",
    "x:02,y:06",
    "x:06,y:14",
    "x:04,y:23",
    "x:-5,y:-4",
    "x:-1,y:-3",
    "x:-2,y:-6",
    "x:-7,y:-1",
    "x:05,y:10",
    "x:06,y:19",
    "x:00,y:13",
    "x:10,y:20"]
M = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
setA = set()

# Error Handling
try:
    if k > len(M):
        raise ValueError("k cannot be greater than M!")
except ValueError as e:
    print(e)
    
for i in range(0, k):
    choice = random.choice(M)
    if choice not in setA:
        setA.add(choice)
    else:
        # this ensures that we will always have 'k' indices, python sets by default will NOT throw errors if we
        # attempt to insert duplicate elements.
        while choice in setA:
            choice = random.choice(M)
        setA.add(choice)

thresholdIndex = random.choice(list(setA))
print(setA)
print(thresholdIndex)



{11, 12, 7}
11


In [3]:
"""
import numpy as np
# Import Qiskit
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi
import math

def printCircuit(circuit):
    #simulate(circuit)
    print(circuit.state)
    
def inversionAboutMean(circuit, n_qubits):
 
    vector = np.array(circuit.state).reshape(-1,1)
    outer_product = np.outer(vector,vector.conj())
    #identity = np.eye(outer_product.shape[0])
    print(vector)
    print(vector.conj())
    print(outer_product * 2/4)

    #operator = 2/4 * identity
    #print(operator*circuit.state)

    print("Inside func: ")
    printCircuit(circuit)
    for qubit in circuit.qubits:
        gate.Hadamard(qubit)
    printCircuit(circuit)
    
    for qubit in circuit.qubits:
        gate.X(qubit)
    
    gate.Hadamard(circuit.qubits[-1])
    
    # Toffoli Operation
    for i in range(n_qubits - 1):
        gate.CX(circuit.qubits[i], circuit.qubits[-1])
    
    gate.Hadamard(circuit.qubits[-1])
    for qubit in circuit.qubits:
        gate.X(qubit)
    for qubit in circuit.qubits:
        gate.Hadamard(qubit)
    
    print("post operations")
     

Goal of Oracle:
Inputted Index: i
Inputted Index corresponds to value: F[i]
Look for F[q] > F[i] where q is not in set A.
Returns 1 if it is value we are looking for, 0 otherwise   

def oracle(circuit, z):
    combinations = {"000":0,"001":1,"010":2,"011":3,"100":4,"101":5,"110":6,"111":7}
    #combinations = {"00":0, "01":1, "10":2, "11":3}
    new_state = circuit.state
    new_state[combinations[z]] = circuit.state[combinations[z]] * -1
    circuit.set_state(new_state, force=True)
    return circuit

def GroversAlgorithm(n_qubits, z):
    circuit = Circuit(n_qubits)
 
    with circuit.context as (qubits, cbits):
        inputRegister = qubits[0:n_qubits-1]
        outputRegister = QuantumRegister(data=[qubits[-1]])
       
        # Apply Hadmards to all the qubits in input register.
        for qubit in inputRegister:
            gate.Hadamard(qubit)
        
        gate.Hadamard(outputRegister.data)
        simulate(circuit)
       
        
        queries = math.ceil(math.sqrt(2**n_qubits))
        for i in range(queries):
            circuit = oracle(circuit, z)
            
            print(circuit.state)
            
            inversionAboutMean(circuit, n_qubits)
            simulate(circuit)
            print(circuit.state)
       
        
"""

In [167]:
import numpy as np

from qiskit import *
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi
import math

def oracle(circuit, quantum_register, classical_register, target, n):
    circuit.barrier()
    
    for i in range(n):
 
        if target[i] == '0':
            
            circuit.x(i)

   
    circuit.mcx(list(range(n)),n)
    
    
    for i in range(n):
        if target[i] == '0':
   
            circuit.x(i)
            
    circuit.barrier()
    return circuit
    
def inversionAboutMean(circuit, quantum_register, n):
    
    for i in range(n):
        circuit.h(i)
        circuit.x(i)
        
    circuit.mcx(list(range(n)),n)

    for i in range(n):
        circuit.x(i)
        circuit.h(i)
        
       
    return circuit

def groversAlgorithm(target):
    n_qubits = len(target)
    quantum_register = QuantumRegister(n_qubits+1)
    classical_register = ClassicalRegister(n_qubits+1)
    circuit = QuantumCircuit(quantum_register, classical_register)
    queries = math.ceil(2**(n_qubits/2))
    
  
    # Hadamard on all qubits
    for i in range(n_qubits):
        circuit.h(i)
    
    for iteration in range(queries):
        
        circuit = oracle(circuit, quantum_register, classical_register, target, n_qubits)
        
        circuit = inversionAboutMean(circuit, quantum_register, n_qubits)
    
    circuit.measure(quantum_register, classical_register)
    
    #print(circuit)
    simulator = AerSimulator()
    transpiled_circuit = transpile(circuit, simulator)

    job = simulator.run(transpiled_circuit, shots=1024).result()
    
    counts = job.get_counts(transpiled_circuit)
   
    
    return counts, circuit
# 000, 010
target = "1000000000000000"
n_qubits = len(target)
counts, circuit = groversAlgorithm(target)

processed_counts = {}
for result, count in counts.items():

    outcome = result[::-1][0:n_qubits]
    
    if outcome in processed_counts:
        processed_counts[outcome] += count
    else:
        processed_counts[outcome] = count
processed_counts = dict(sorted(processed_counts.items()))
print(processed_counts)

measurement = max(counts, key=counts.get)
measurement = measurement[::-1][0:n_qubits]

if measurement == target:
    print("Found Target!")
else:
    print("Failed to find target!")
#plot_histogram(processed_counts, title='Bell-State counts')
#circuit.draw('mpl')

{'0000000010010010': 1, '0000000010010011': 1, '0000000011010000': 1, '0000000100100111': 1, '0000000100111011': 1, '0000000101111010': 1, '0000000110001001': 1, '0000000111110111': 1, '0000001000100011': 1, '0000001011100001': 1, '0000001100001110': 1, '0000001100110000': 1, '0000001101011011': 1, '0000001101110111': 1, '0000010110011010': 1, '0000011001011110': 1, '0000011111100111': 1, '0000100011011000': 1, '0000100101100000': 1, '0000100111001100': 1, '0000100111110101': 1, '0000101000000011': 1, '0000101100001011': 1, '0000101101111110': 1, '0000101111011000': 1, '0000110000011010': 1, '0000110001000111': 1, '0000110010011001': 1, '0000110100000100': 1, '0000110110110000': 1, '0000111010010111': 1, '0000111010011111': 1, '0000111011110100': 1, '0000111110010011': 1, '0001000000010000': 1, '0001000011000010': 1, '0001001001100010': 1, '0001001010101101': 1, '0001001100101001': 1, '0001001100101111': 1, '0001001111110001': 1, '0001010001001000': 1, '0001010010011001': 1, '000101001